In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.9/626.9 kB 9.9 MB/s eta 0:00:00


In [2]:
#requirements
import glob
import shutil
import yaml
import os
import cv2
import re
from tqdm import tqdm
import math
import random
import ultralytics
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/cj_data'

In [5]:
#Unzip Files
# %cd /content/drive/MyDrive/cj_data/data
# !unzip "data.zip"
# #!unzip "labels.zip"
# %cd /content

/content/drive/MyDrive/cj_data/data
Archive:  data.zip
   creating: Data/
   creating: Data/image/
  inflating: Data/image/img_0101.jpg  
  inflating: Data/image/img_0102.jpg  
  inflating: Data/image/img_0103.jpg  
  inflating: Data/image/img_0201.jpg  
  inflating: Data/image/img_0202.jpg  
  inflating: Data/image/img_0203.jpg  
  inflating: Data/image/img_0204.jpg  
  inflating: Data/image/img_0205.jpg  
  inflating: Data/image/img_0301.jpg  
  inflating: Data/image/img_0302.jpg  
  inflating: Data/image/img_0303.jpg  
  inflating: Data/image/img_0304.jpg  
  inflating: Data/image/img_0305.jpg  
  inflating: Data/image/img_0306.jpg  
  inflating: Data/image/img_0401.jpg  
  inflating: Data/image/img_0402.jpg  
  inflating: Data/image/img_0403.jpg  
  inflating: Data/image/img_0404.jpg  
  inflating: Data/image/img_0405.jpg  
  inflating: Data/image/img_0406.jpg  
  inflating: Data/image/img_0407.jpg  
  inflating: Data/image/img_0408.jpg  
  inflating: Data/image/img_0501.jpg  
  in

In [10]:
# #data Unzip 한 이후 image 폴더만 옮길경우
# img_folder = f'{path}/data/Data/image'
# dis = f'{path}/data'
# shutil.move(img_folder, dis)

'/content/drive/MyDrive/cj_data/data/image'

In [11]:
# shutil.rmtree('/content/drive/MyDrive/cj_data/data/Data')

In [ ]:
#Unzip label Files
# %cd /content/drive/MyDrive/cj_data/data
# !unzip "labels.zip"
# %cd /content

In [12]:
image_len = (glob.glob(f'{path}/data/image/*.jpg'))
len(image_len)

596

In [13]:
labels_len = (glob.glob(f'{path}/data/labels/*.txt'))
len(labels_len)

596

In [14]:
validation_len = (glob.glob(f'{path}/data/validation/**/*.jpg'))
len(validation_len)


45

### making Yaml File

In [15]:
with open(f'{path}/data/labeling.txt', 'r', encoding='utf-8') as la:
    next(la)
    lines = la.readlines()
    classes = [(line.strip().split('\t')) for line in lines]


In [16]:
labels_lst = []
labels = []
for cl in classes:
    labels_lst.append((str(cl[0])+ '_' + cl[1]))
    labels.append(cl[0])

In [17]:
data_yaml = {
    'nc':len(classes),
    'names': labels,
    'train': '/content/drive/MyDrive/cj_data/data/train',
    'val': '/content/drive/MyDrive/cj_data/data/val',
    'test': '/content/drive/MyDrive/cj_data/data/test'
}
with open(f'{path}/data/data.yaml', 'w', encoding='utf-8') as f:
    yaml.dump(data_yaml, f,allow_unicode=True)

In [18]:
!cat /content/drive/MyDrive/cj_data/data/data.yaml

names:
- '01'
- '02'
- '03'
- '04'
- '05'
- '06'
- '07'
- 08
- 09
- '10'
- '11'
- '13'
- '14'
- '15'
- '16'
- '17'
- '18'
- '19'
- '20'
- '21'
- '23'
- '24'
- '25'
- '26'
- '29'
- '30'
- '31'
- '32'
- '34'
- '35'
- '37'
- '38'
- '40'
- '41'
- '42'
- '43'
- '44'
- '45'
- '46'
- '47'
- '48'
- '49'
- '50'
- '51'
- '52'
- '53'
- '54'
- '55'
- '56'
- '57'
- '58'
- '59'
- '60'
- '61'
- '62'
- '63'
- '64'
- '65'
- '67'
- '68'
- '69'
- '70'
- '72'
- '73'
- '74'
- '75'
- '76'
- '77'
- '79'
- '80'
- '81'
- '82'
- '83'
- '84'
- '85'
- '86'
- '87'
- '88'
- '89'
- '90'
- '91'
- '92'
- '93'
- '95'
- '96'
- '97'
- '98'
- '99'
- '100'
- '101'
- '102'
- '103'
- '105'
- '107'
- '108'
- '109'
- '110'
- '111'
- '113'
- '114'
nc: 100
test: /content/drive/MyDrive/cj_data/data/test
train: /content/drive/MyDrive/cj_data/data/train
val: /content/drive/MyDrive/cj_data/data/val


### Move Files

In [19]:
# cj_data
# ㄴ Data
#     ㄴimage# 기존 이미지 데이터 모아놓은 폴더
#     ㄴimg   #labels별로 이미지 분리하여 폴더에 저장
#     ㄴtrain #train 이미지를 저장
#     ㄴval   #val 이미지를 저장
#     ㄴtest

In [20]:
labels = []
for cl in data_yaml['names']:
    labels.append(cl)

In [21]:
#Move Labels
#bounding box 형식 labels max min max min

In [22]:
labels_len = (glob.glob(f'{path}/data/labels/*.txt'))
labels_len
'/content/drive/MyDrive/cj_data/data/labels/img_0101_jpg.rf.a1c83dbccf127954bf50c1a0d87a1984.txt'

'/content/drive/MyDrive/cj_data/data/labels/img_0101_jpg.rf.a1c83dbccf127954bf50c1a0d87a1984.txt'

In [23]:
def rename_texts(texts_list):
    for file in tqdm(texts_list):
        file_name = os.path.basename(file)
        file_name = file_name.split('.')[0][:-4]
        img = cv2.imread(f'/content/drive/MyDrive/cj_data/data/image/{file_name}.jpg')
        img_height, img_width, _ = img.shape
        cv2.destroyAllWindows()
        with open(file, 'r') as f:
            lines_list = []
            line = f.readlines()
            x1, y1, x2, y2, x3, y3, x4, y4, class_id, _ = line[0].split(' ')
            # OBB의 중심 좌표 계산
            x_center = (float(x1) + float(x2) + float(x3) + float(x4)) / 4
            y_center = (float(y1) + float(y2) + float(y3) + float(y4)) / 4

            # OBB의 너비와 높이 계산
            obb_width = math.sqrt((float(x2) - float(x1)) ** 2 + (float(y2) - float(y1)) ** 2)
            obb_height = math.sqrt((float(x3) - float(x2)) ** 2 + (float(y3) - float(y2)) ** 2)

            # YOLO 좌표 계산
            yolo_x = x_center / img_width
            yolo_y = y_center / img_height
            yolo_width = obb_width / img_width
            yolo_height = obb_height / img_height

            # YOLO 좌표를 문자열 형식으로 반환
            yolo_coords = f"{class_id} {yolo_x} {yolo_y} {yolo_width} {yolo_height}"
        with open(f'/content/drive/MyDrive/cj_data/data/re_labels/{file_name}.txt', 'w') as f:
            f.write(yolo_coords)
    print('\n')
    print('finish')

In [24]:
#Checking files in dir, rename_texts
if len(os.listdir('/content/drive/MyDrive/cj_data/data/re_labels')) != len(labels_len):
    rename_texts(labels_len)

100%|██████████| 596/596 [04:45<00:00,  2.09it/s]



finish


In [40]:
###폴더 초기화
shutil.rmtree('/content/drive/MyDrive/cj_data/data/test')
shutil.rmtree('/content/drive/MyDrive/cj_data/data/train')
shutil.rmtree('/content/drive/MyDrive/cj_data/data/val')

#Making Datasets Folder
if not os.path.exists('/content/drive/MyDrive/cj_data/data/train'):
    os.makedirs('/content/drive/MyDrive/cj_data/data/train')

if not os.path.exists('/content/drive/MyDrive/cj_data/data/val'):
    os.makedirs('/content/drive/MyDrive/cj_data/data/val')

if not os.path.exists('/content/drive/MyDrive/cj_data/data/test'):
    os.makedirs('/content/drive/MyDrive/cj_data/data/test')


In [41]:
#Divide train, val datasets
path = '/content/drive/MyDrive/cj_data/data'
images_path = sorted(glob.glob(f'{path}/image/*.jpg'))
texts_path = sorted(glob.glob(f'{path}/re_labels/*.txt'))
print(len(images_path), len(texts_path))
train_images, valid_images, train_txts, valid_txts = train_test_split(images_path, texts_path, test_size = 0.1, random_state=74)

596 596


In [42]:
#Move FIles to Folder
train_folder = f'{path}/train'
valid_folder = f'{path}/val'
test_folder = f'{path}/test'

train_images.sort()
valid_images.sort()
train_txts.sort()
valid_txts.sort()

for train_image in tqdm(train_images, desc='Copying train images'):
    shutil.copy(train_image, train_folder)
for train_txt in tqdm(train_txts, desc='Copying train annotations'):
    shutil.copy(train_txt, train_folder)


for valid_image in tqdm(valid_images, desc='Copying valid images'):
    shutil.copy(valid_image, valid_folder)
for valid_txt in tqdm(valid_txts, desc='Copying valid annotations'):
    shutil.copy(valid_txt, valid_folder)


Copying valid annotations: 100%|██████████| 60/60 [00:00<00:00, 99.54it/s]


In [43]:
#Check
len_images = sorted(glob.glob(f'{path}/train/*.jpg'))
len_texts = sorted(glob.glob(f'{path}/train/*.txt'))

len_images_v = sorted(glob.glob(f'{path}/val/*.jpg'))
len_texts_v = sorted(glob.glob(f'{path}/val/*.txt'))
print(len(len_images), len(len_texts))
print(len(len_images_v), len(len_texts_v))

536 536
60 60


### Train Yolov8